In [1]:
import os,sys
#sys.path.append('/work/qdiff/mo_utils')

In [ ]:
from mo_utils.utils.tmux_utils import get_session_list,tmux_session,get_session_name,kill_session
from pathlib import Path

libtmux not installed ??


In [ ]:
get_session_list()

[Session($0 0),
 Session($15 sd_quantize_wb=4_gpu_7),
 Session($14 sd_quantize_wb=8_gpu_2),
 Session($13 sd_quantize_wb=8_gpu_6)]

In [4]:
#kill_session()

before kill sessions=[Session($0 0), Session($15 sd_quantize_wb=4_gpu_7), Session($14 sd_quantize_wb=8_gpu_2), Session($13 sd_quantize_wb=8_gpu_6)]
after kill sessions=[Session($0 0)]


In [19]:
task = 'quantize'
gpu = 0
prompt = "a puppy wearing a hat" 
weight_bit = 4
symmetric_weight = False
resume_w = False
bs = 8
outdir= "output_quantization"
#quantized_ckpt_path = '/home/nadavg/q-diffusion/output_quantization/2025-01-22-16-40-17/wc_ckpt.pth'
quantized_ckpt_path = '/home/nadavg/q-diffusion/output_quantization2/2025-01-24-00-49-55/wc_ckpt.pth'
cali_data_path='/fastdata/users/nadavg/sd/qdiff/sd_coco-s75_sample1024_allst.pt'

In [20]:
cmd=(f"python scripts/txt2img.py --prompt '{prompt}' --plms --cond --ptq --weight_bit {weight_bit} --quant_mode qdiff "+
    f"--quant_act --act_bit 8 --cali_st 25 --cali_batch_size {bs} --cali_n 128 --no_grad_ckpt --split --running_stat "+
    f"--sm_abit 16 --cali_data_path {cali_data_path} --outdir {outdir}"+
    symmetric_weight*" --symmetric_weight "+
    resume_w*f"--resume_w --cali_ckpt {quantized_ckpt_path}"
    )


In [21]:
cmd

"python scripts/txt2img.py --prompt 'a puppy wearing a hat' --plms --cond --ptq --weight_bit 4 --quant_mode qdiff --quant_act --act_bit 8 --cali_st 25 --cali_batch_size 8 --cali_n 128 --no_grad_ckpt --split --running_stat --sm_abit 16 --cali_data_path /fastdata/users/nadavg/sd/qdiff/sd_coco-s75_sample1024_allst.pt --outdir output_quantization"

In [22]:
inst_list = [f'cd {Path.home() / "q-diffusion"}',
             f'conda activate qdiff',
             f'export CUDA_VISIBLE_DEVICES={gpu}',
             cmd]

In [23]:
inst_list

['cd /home/nadavg/q-diffusion',
 'conda activate qdiff',
 'export CUDA_VISIBLE_DEVICES=0',
 "python scripts/txt2img.py --prompt 'a puppy wearing a hat' --plms --cond --ptq --weight_bit 4 --quant_mode qdiff --quant_act --act_bit 8 --cali_st 25 --cali_batch_size 8 --cali_n 128 --no_grad_ckpt --split --running_stat --sm_abit 16 --cali_data_path /fastdata/users/nadavg/sd/qdiff/sd_coco-s75_sample1024_allst.pt --outdir output_quantization"]

In [24]:
sess_name = get_session_name(f'sd_{task}_wb={weight_bit}_wsym={symmetric_weight}_gpu_{gpu}')
sess_name

'sd_quantize_wb=4_wsym=False_gpu_0'

In [25]:
tmux_session(sess_name,inst_list)

tmux attach -t "sd_quantize_wb=4_wsym=False_gpu_0"


'sd_quantize_wb=4_wsym=False_gpu_0'